## Imports and Config

In [76]:
import gymnasium as gym
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import random
from collections import namedtuple, deque
import datetime

import matplotlib.pyplot as plt

In [77]:
BUFFER_SIZE = 100000
BATCH_SIZE = 64
GAMMA = 0.99
LR = 0.0005
UPDATE_EVERY = 20

In [78]:
env = gym.make('CartPole-v1')

state_shape = env.observation_space.shape[0]
n_actions = env.action_space.n      

state = env.reset(seed=0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

## Dueling DQN

### QNetwork

In [79]:
class QNetwork(nn.Module):

    def __init__(
        self,
        state_size: int,
        action_size: int,
        seed: int = 0,
        fc1_units: int = 128,
        fc2_units: int = 64,
    ):
        """Initialize parameters and build model."""
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.value_layer = nn.Linear(fc2_units, 1)
        self.advantage_layer = nn.Linear(fc2_units, action_size)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        raise NotImplementedError

In [80]:
class QNetwork1(QNetwork):

    def __init__(
        self,
        state_size: int,
        action_size: int,
        seed: int = 0,
        fc1_units: int = 128,
        fc2_units: int = 64,
    ):
        """Initialize parameters and build model."""
        super(QNetwork1, self).__init__(
            state_size, action_size, seed, fc1_units, fc2_units
        )

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Build a network that maps state -> action values."""

        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))

        value = self.value_layer(x)
        advantage = self.advantage_layer(x)

        q = value + (advantage - advantage.mean())
        return q

In [ ]:
class QNetwork2(QNetwork):

    def __init__(
        self,
        state_size: int,
        action_size: int,
        seed: int = 0,
        fc1_units: int = 128,
        fc2_units: int = 64,
    ):
        """Initialize parameters and build model."""
        super(QNetwork1, self).__init__(
            state_size, action_size, seed, fc1_units, fc2_units
        )

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Build a network that maps state -> action values."""

        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))

        value = self.value_layer(x)
        advantage = self.advantage_layer(x)

        q = value + (advantage - advantage.max())
        return q

### Replay Buffer

In [81]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object."""

        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

## Agent

In [82]:
class Agent:

    def __init__(self, state_size: int, action_size: int, seed: int):
        """Agent-Environment Interaction"""
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        """ Q-Network """
        self.qnetwork_local = QNetwork1(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork1(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        """ Replay memory"""
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

        """ Initialize time step (for updating every UPDATE_EVERY steps) """
        self.t_step = 0

    def step(
        self,
        state: np.ndarray,
        action: int,
        reward: float,
        next_state: np.ndarray,
        done: bool,
    ):
        """Save experience in replay memory"""
        self.memory.add(state, action, reward, next_state, done)

        """ If enough samples are available in memory, get random subset and learn """
        if len(self.memory) >= BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

        """ Update the target network every UPDATE_EVERY steps """
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())

    def act(self, state: np.ndarray, eps: float = 0.0) -> int:

        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        """ Epsilon-greedy action selection """
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences: namedtuple, gamma: float):
        """+E EXPERIENCE REPLAY PRESENT"""
        states, actions, rewards, next_states, dones = experiences

        """ Get max predicted Q values (for next states) from target model """
        Q_targets_next = (
            self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        )

        """ Compute Q targets for current states """
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        """ Get expected Q values from local model """
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        """ Compute loss """
        loss = F.mse_loss(Q_expected, Q_targets)

        """ Minimize the loss """
        self.optimizer.zero_grad()
        loss.backward()

        """ Gradient Clipping """
        """ +T TRUNCATION PRESENT """
        for param in self.qnetwork_local.parameters():
            # check if some parameter has gradient none, if so print the name of the parameter
            if param.grad is None:
                print(param.names, "has gradient None", param.shape)

        self.optimizer.step()

In [83]:
class Trainer(Agent):

    def __init__(self, state_size: int, action_size: int, seed: int):
        super(Trainer, self).__init__(state_size, action_size, seed)

    def train(
        self,
        n_episodes: int = 2000,
        max_t: int = 1000,
        eps_start: float = 1.0,
        eps_end: float = 0.01,
        eps_decay: float = 0.995,
    ):
        scores_window = deque(maxlen=100)
        scores_list = []
        eps = eps_start

        for i_episode in range(1, n_episodes + 1):
            state, _ = env.reset()
            score = 0
            
            for t in range(max_t):
                action = self.act(state, eps)
                next_state, reward, done, _, _ = env.step(action)
                self.step(state, action, reward, next_state, done)
                state = next_state
                score += reward
                if done:
                    break
        
            scores_window.append(score)
            scores_list.append(score)

            eps = max(eps_end, eps_decay * eps)

            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
            
            if i_episode % 100 == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            if np.mean(scores_window) >= 195.0:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode - 100, np.mean(scores_window)))
                torch.save(self.qnetwork_local.state_dict(), 'checkpoint.pth')
                break
        
        return scores_list

In [84]:
begin_time = datetime.datetime.now()

trainer = Trainer(state_shape, n_actions, seed=0)
scores = trainer.train()

time_elapsed = datetime.datetime.now() - begin_time

print('Time Elapsed: ', time_elapsed)

Episode 7	Average Score: 16.86

Episode 100	Average Score: 46.94
Episode 200	Average Score: 165.33
Episode 221	Average Score: 196.96
Environment solved in 121 episodes!	Average Score: 196.96
Time Elapsed:  0:00:53.058109
